# Classification avec Transformer (Encoder seul)

In [13]:
# !pip install dill

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import sys
sys.path.append('..')
from utils import *

## Chargement et prétraitement des données (seulement en anglais)

In [2]:
toots = pd.read_csv('../data/100k_en_toots_labeled.csv').dropna()

ys, nb_ys = np.unique(toots['y'], return_counts=True)
print('Taille des toots(toutes les langues)', len(toots))
print(f'Nb pos({ys[1]}): {nb_ys[1]} -- Nb neg({ys[0]}): {nb_ys[0]} -- total: {nb_ys.sum()}')

# toots en anglais
itoots_en = toots['language'] == 'en'
toots_en = toots[itoots_en]
toots_en = toots_en[['content', 'y']]

ys, nb_ys = np.unique(toots_en['y'], return_counts=True)
print('Taille des toots en anglais', len(toots_en))
print(f'Nb pos({ys[1]}): {nb_ys[1]} -- Nb neg({ys[0]}): {nb_ys[0]} -- total: {nb_ys.sum()}')

toots_en.head()

Taille des toots(toutes les langues) 93273
Nb pos(1): 1569 -- Nb neg(0): 91704 -- total: 93273
Taille des toots en anglais 47465
Nb pos(1): 1011 -- Nb neg(0): 46454 -- total: 47465


,content,y
0,Chikmagalur Tourist Places: Your Ultimate Guid...,0
1,"Dancing Adélie Penguins, McMurdo Sound, Antar...",0
2,2 Macdonald trip leaving Burrard Station @ Bay...,0
3,"Here you go seekers, some more good music (the...",0
11,The Future of Nuclear Energy in a Carbon-Const...,0


## Prétraitement et sauvegarde des embeddings (20 minutes)

In [6]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Encoder les toots
# embeddings = model.encode(toots_en['content'].tolist())

Batches:   0%|          | 0/1484 [00:00<?, ?it/s]

In [14]:
# y = toots_en['y'].to_list()
# save_object('../data/embedding_100k_en_toots_labeled_eng.dill', [(embi,yi) for embi,yi in zip(embeddings, y)])

## Charger les embeddings déjà prétraités et enregistrés

mettre les données des embeddings dans: src/data/embedding_100k_en_toots_labeled_eng.csv

In [25]:
data = load_object('../data/embedding_100k_en_toots_labeled_eng.csv')

## DataLoader

In [117]:
class MyData(Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

dataset = MyData(data)

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, test_indices = next(iter(splitter.split(dataset, [label for _, label in data])))

# Use the indices to create training and testing datasets
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# equilibrer train
ipos = [i for i,(d,y) in enumerate(train_dataset)  if y==1]
ineg = [i for i,(d,y) in enumerate(train_dataset)  if y==0]
np.random.shuffle(ineg)
train_pos = torch.utils.data.Subset(train_dataset, ipos)
train_neg = torch.utils.data.Subset(train_dataset, ineg[:len(ipos)])
train_dataset_eq = torch.utils.data.ConcatDataset([train_pos,train_neg])
# equilibrer test
ipos = [i for i,(d,y) in enumerate(test_dataset)  if y==1]
ineg = [i for i,(d,y) in enumerate(test_dataset)  if y==0]
np.random.shuffle(ineg)
train_pos = torch.utils.data.Subset(test_dataset, ipos)
train_neg = torch.utils.data.Subset(test_dataset, ineg[:len(ipos)])
test_dataset_eq = torch.utils.data.ConcatDataset([train_pos,train_neg])

# Create DataLoaders for iterating over the training and testing sets
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_dataloader_eq = DataLoader(train_dataset_eq, batch_size=batch_size, shuffle=True)
test_dataloader_eq = DataLoader(test_dataset_eq, batch_size=batch_size, shuffle=False)

In [100]:
class Net(nn.Module):
  def __init__(self, dim=384):
    super(Net, self).__init__()
    
    self.main = nn.Sequential(
      nn.Linear(dim, 2 * dim),
      nn.ReLU(),
      nn.Linear(2 * dim, 4 * dim),
      nn.ReLU(),
      nn.Linear(4 * dim, 2 * dim),
      nn.ReLU(),
      nn.Linear(2 * dim, dim),
      nn.ReLU(),
      nn.Linear(dim, 1),
      nn.Sigmoid()
    )
  
  def forward(self, x):
    return self.main(x)

class F1Loss(nn.Module):
  def __init__(self, epsilon=1e-7):
    super(F1Loss, self).__init__()
    self.epsilon = epsilon

  def forward(self, y_true, y_pred):
    # Calculez les vrais positifs, faux positifs et faux négatifs
    tp = torch.sum(y_true * y_pred)
    fp = torch.sum((1 - y_true) * y_pred)
    fn = torch.sum(y_true * (1 - y_pred))

    # Calculez la précision, le rappel et le F1 score
    precision = tp / (tp + fp + self.epsilon)
    recall = tp / (tp + fn + self.epsilon)
    f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)

    # Utilisez 1 - F1 comme la perte (car PyTorch minimise)
    loss = 1 - f1
    return loss

## Apprentissage avec des classes non équilibrées avec MSELoss

In [136]:
net = Net()
optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)
# criterion = F1Loss()
criterion = nn.MSELoss()

epochs = 7
train_loss = []
test_loss = []

In [137]:
for epoch in range(epochs):
  print(f"Epoch [{(epoch+1):4d}/{epochs:4d}] ")
  lep = []
  net.train()
  for batch_idx, (batch, y) in enumerate(train_dataloader):

    yhat = net(batch).squeeze(dim=1)
    y = y.float()
    loss = criterion(y, yhat)
    
    net.zero_grad()
    loss.backward()
    optimizer.step()
    
    # save loss train
    lep.append(loss.detach().numpy())
    
    
    yhat_label = torch.tensor([1 if yihat > 0.5 else 0 for yihat in yhat])
    ipos, ineg = y == 1, y == 0
    y_pos, y_neg = y[ipos], y[ineg]
    yhat_pos, yhat_neg = yhat_label[ipos], yhat_label[ineg]
    if len(y_pos)>0:
      pred_pos = (yhat_pos == y_pos).sum()/ len(y_pos)
    if len(y_neg)>0:
      pred_neg = (yhat_neg == y_neg).sum()/ len(y_neg)
    print(
      f"\ttrain batch [{(batch_idx+1):4d}/{len(train_dataloader):4d}] - "
      f"Loss : {loss:.4f} - "
      f"TP/P : {(100*pred_pos):.2f} - "
      f"TN/N : {(100*pred_neg):.2f}", end="\r"
    )
  print(
    f"\ttrain batch [{(batch_idx+1):4d}/{len(train_dataloader):4d}] - "
    f"Loss : {loss:.4f} - "
    f"TP/P : {(100*pred_pos):.2f} - "
    f"TN/N : {(100*pred_neg):.2f}"
  )
  train_loss.append(np.mean(lep))
  
  lep = []
  net.eval()
  for batch_idx, (batch, y) in enumerate(test_dataloader):

    yhat = net(batch).squeeze(dim=1)
    loss = criterion(y, yhat)
    lep.append(loss.detach().numpy())
    
    yhat_label = torch.tensor([1 if yihat > 0.5 else 0 for yihat in yhat])
    ipos, ineg = y == 1, y == 0
    y_pos, y_neg = y[ipos], y[ineg]
    yhat_pos, yhat_neg = yhat_label[ipos], yhat_label[ineg]
    if len(y_pos)>0:
      pred_pos = (yhat_pos == y_pos).sum()/ len(y_pos)
    if len(y_neg)>0:
      pred_neg = (yhat_neg == y_neg).sum()/ len(y_neg)
    print(
      f"\ttest batch [{(batch_idx+1):4d}/{len(test_dataloader):4d}] - "
      f"Loss : {loss:.4f} - "
      f"TP/P : {(100*pred_pos):.2f} - "
      f"TN/N : {(100*pred_neg):.2f}", end="\r"
    )
  print(
    f"\ttest batch [{(batch_idx+1):4d}/{len(test_dataloader):4d}] - "
    f"Loss : {loss:.4f} - "
    f"TP/P : {(100*pred_pos):.2f} - "
    f"TN/N : {(100*pred_neg):.2f}"
  )
  test_loss.append(np.mean(lep))

Epoch [   1/   7] 
	train batch [1187/1187] - Loss : 0.0056 - TP/P : 0.00 - TN/N : 100.000
	test batch [ 297/ 297] - Loss : 0.0017 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   2/   7] 
	train batch [1187/1187] - Loss : 0.0001 - TP/P : 0.00 - TN/N : 100.0000
	test batch [ 297/ 297] - Loss : 0.0003 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   3/   7] 
	train batch [1187/1187] - Loss : 0.0023 - TP/P : 100.00 - TN/N : 100.00
	test batch [ 297/ 297] - Loss : 0.0000 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   4/   7] 
	train batch [1187/1187] - Loss : 0.0000 - TP/P : 33.33 - TN/N : 100.000
	test batch [ 297/ 297] - Loss : 0.0000 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   5/   7] 
	train batch [1187/1187] - Loss : 0.0094 - TP/P : 0.00 - TN/N : 100.0000
	test batch [ 297/ 297] - Loss : 0.0000 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   6/   7] 
	train batch [1187/1187] - Loss : 0.0013 - TP/P : 100.00 - TN/N : 100.00
	test batch [ 297/ 297] - Loss : 0.0000 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   7/   7]

### Taux de bonnes prédiction dans train

In [155]:
net.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(train_dataloader):
  
  yhat = net(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 99.10%
Taux de prediction sur les pos: 68.85%
Taux de prediction sur les neg: 99.76%
F1 score: 76.46%

accuracy score: 99.10%
precision score: 68.85%
recall score: 85.96%
f1 score: 76.46%


### Taux de bonnes prédiction dans test

In [156]:
net.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(test_dataloader):
  
  yhat = net(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 96.99%
Taux de prediction sur les pos: 28.71%
Taux de prediction sur les neg: 98.47%
F1 score: 28.86%

accuracy score: 96.99%
precision score: 28.71%
recall score: 29.00%
f1 score: 28.86%


## Apprentissage avec des classes non équilibrées avec f1_loss

In [157]:
net_f1 = Net()
optimizer = torch.optim.Adam(net_f1.parameters(), lr=3e-4)
criterion = F1Loss()
# criterion = nn.MSELoss()

epochs = 7
train_loss = []
test_loss = []

In [158]:
for epoch in range(epochs):
  print(f"Epoch [{(epoch+1):4d}/{epochs:4d}] ")
  lep = []
  net_f1.train()
  for batch_idx, (batch, y) in enumerate(train_dataloader):

    yhat = net_f1(batch).squeeze(dim=1)
    y = y.float()
    loss = criterion(y, yhat)
    
    net_f1.zero_grad()
    loss.backward()
    optimizer.step()
    
    # save loss train
    lep.append(loss.detach().numpy())
    
    
    yhat_label = torch.tensor([1 if yihat > 0.5 else 0 for yihat in yhat])
    ipos, ineg = y == 1, y == 0
    y_pos, y_neg = y[ipos], y[ineg]
    yhat_pos, yhat_neg = yhat_label[ipos], yhat_label[ineg]
    if len(y_pos)>0:
      pred_pos = (yhat_pos == y_pos).sum()/ len(y_pos)
    if len(y_neg)>0:
      pred_neg = (yhat_neg == y_neg).sum()/ len(y_neg)
    print(
      f"\ttrain batch [{(batch_idx+1):4d}/{len(train_dataloader):4d}] - "
      f"Loss : {loss:.4f} - "
      f"TP/P : {(100*pred_pos):.2f} - "
      f"TN/N : {(100*pred_neg):.2f}", end="\r"
    )
  print(
    f"\ttrain batch [{(batch_idx+1):4d}/{len(train_dataloader):4d}] - "
    f"Loss : {loss:.4f} - "
    f"TP/P : {(100*pred_pos):.2f} - "
    f"TN/N : {(100*pred_neg):.2f}"
  )
  train_loss.append(np.mean(lep))
  
  lep = []
  net_f1.eval()
  for batch_idx, (batch, y) in enumerate(test_dataloader):

    yhat = net_f1(batch).squeeze(dim=1)
    loss = criterion(y, yhat)
    lep.append(loss.detach().numpy())
    
    yhat_label = torch.tensor([1 if yihat > 0.5 else 0 for yihat in yhat])
    ipos, ineg = y == 1, y == 0
    y_pos, y_neg = y[ipos], y[ineg]
    yhat_pos, yhat_neg = yhat_label[ipos], yhat_label[ineg]
    if len(y_pos)>0:
      pred_pos = (yhat_pos == y_pos).sum()/ len(y_pos)
    if len(y_neg)>0:
      pred_neg = (yhat_neg == y_neg).sum()/ len(y_neg)
    print(
      f"\ttest batch [{(batch_idx+1):4d}/{len(test_dataloader):4d}] - "
      f"Loss : {loss:.4f} - "
      f"TP/P : {(100*pred_pos):.2f} - "
      f"TN/N : {(100*pred_neg):.2f}", end="\r"
    )
  print(
    f"\ttest batch [{(batch_idx+1):4d}/{len(test_dataloader):4d}] - "
    f"Loss : {loss:.4f} - "
    f"TP/P : {(100*pred_pos):.2f} - "
    f"TN/N : {(100*pred_neg):.2f}"
  )
  test_loss.append(np.mean(lep))

Epoch [   1/   7] 
	train batch [1187/1187] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0000
	test batch [ 297/ 297] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 95.24050
Epoch [   2/   7] 
	train batch [1187/1187] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 95.00320
	test batch [ 297/ 297] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 95.24000
Epoch [   3/   7] 
	train batch [1187/1187] - Loss : 1.0000 - TP/P : 100.00 - TN/N : 95.000
	test batch [ 297/ 297] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0030
Epoch [   4/   7] 
	train batch [1187/1187] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0000
	test batch [ 297/ 297] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0070
Epoch [   5/   7] 
	train batch [1187/1187] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0000
	test batch [ 297/ 297] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   6/   7] 
	train batch [1187/1187] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0000
	test batch [ 297/ 297] - Loss : 1.0000 - TP/P : 0.00 - TN/N : 100.0000
Epoch [   7/   7

### Taux de bonnes prédiction dans train

In [159]:
net_f1.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(train_dataloader):
  
  yhat = net_f1(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 96.50%
Taux de prediction sur les pos: 28.55%
Taux de prediction sur les neg: 97.98%
F1 score: 25.80%

accuracy score: 96.50%
precision score: 28.55%
recall score: 23.52%
f1 score: 25.80%


### Taux de bonnes prédiction dans test

In [161]:
net_f1.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(test_dataloader):
  
  yhat = net_f1(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 96.23%
Taux de prediction sur les pos: 23.76%
Taux de prediction sur les neg: 97.80%
F1 score: 21.15%

accuracy score: 96.23%
precision score: 23.76%
recall score: 19.05%
f1 score: 21.15%


## Apprentissage avec des classes équilibrées

In [143]:
net_eq = Net()
optimizer = torch.optim.Adam(net_eq.parameters(), lr=3e-4)
# criterion = F1Loss()
criterion = nn.MSELoss()

epochs = 7
train_loss = []
test_loss = []

In [144]:
for epoch in range(epochs):
  print(f"Epoch [{(epoch+1):4d}/{epochs:4d}] ")
  lep = []
  net_eq.train()
  for batch_idx, (batch, y) in enumerate(train_dataloader_eq):

    yhat = net_eq(batch).squeeze(dim=1)
    y = y.float()
    loss = criterion(y, yhat)
    
    net_eq.zero_grad()
    loss.backward()
    optimizer.step()
    
    # save loss train
    lep.append(loss.detach().numpy())
    
    
    yhat_label = torch.tensor([1 if yihat > 0.5 else 0 for yihat in yhat])
    ipos, ineg = y == 1, y == 0
    y_pos, y_neg = y[ipos], y[ineg]
    yhat_pos, yhat_neg = yhat_label[ipos], yhat_label[ineg]
    if len(y_pos)>0:
      pred_pos = (yhat_pos == y_pos).sum()/ len(y_pos)
    if len(y_neg)>0:
      pred_neg = (yhat_neg == y_neg).sum()/ len(y_neg)
    print(
      f"\ttrain batch [{(batch_idx+1):4d}/{len(train_dataloader_eq):4d}] - "
      f"Loss : {loss:.4f} - "
      f"TP/P : {(100*pred_pos):.2f} - "
      f"TN/N : {(100*pred_neg):.2f}", end="\r"
    )
  print(
    f"\ttrain batch [{(batch_idx+1):4d}/{len(train_dataloader_eq):4d}] - "
    f"Loss : {loss:.4f} - "
    f"TP/P : {(100*pred_pos):.2f} - "
    f"TN/N : {(100*pred_neg):.2f}"
  )
  train_loss.append(np.mean(lep))
  
  lep = []
  net_eq.eval()
  for batch_idx, (batch, y) in enumerate(test_dataloader_eq):

    yhat = net_eq(batch).squeeze(dim=1)
    loss = criterion(y, yhat)
    lep.append(loss.detach().numpy())
    
    yhat_label = torch.tensor([1 if yihat > 0.5 else 0 for yihat in yhat])
    ipos, ineg = y == 1, y == 0
    y_pos, y_neg = y[ipos], y[ineg]
    yhat_pos, yhat_neg = yhat_label[ipos], yhat_label[ineg]
    if len(y_pos)>0:
      pred_pos = (yhat_pos == y_pos).sum()/ len(y_pos)
    if len(y_neg)>0:
      pred_neg = (yhat_neg == y_neg).sum()/ len(y_neg)
    print(
      f"\ttest batch [{(batch_idx+1):4d}/{len(test_dataloader_eq):4d}] - "
      f"Loss : {loss:.4f} - "
      f"TP/P : {(100*pred_pos):.2f} - "
      f"TN/N : {(100*pred_neg):.2f}", end="\r"
    )
  print(
    f"\ttest batch [{(batch_idx+1):4d}/{len(test_dataloader_eq):4d}] - "
    f"Loss : {loss:.4f} - "
    f"TP/P : {(100*pred_pos):.2f} - "
    f"TN/N : {(100*pred_neg):.2f}"
  )
  test_loss.append(np.mean(lep))

Epoch [   1/   7] 
	train batch [  51/  51] - Loss : 0.1205 - TP/P : 100.00 - TN/N : 81.82
	test batch [  13/  13] - Loss : 0.0780 - TP/P : 90.00 - TN/N : 90.00
Epoch [   2/   7] 
	train batch [  51/  51] - Loss : 0.0956 - TP/P : 90.00 - TN/N : 87.500
	test batch [  13/  13] - Loss : 0.1066 - TP/P : 90.00 - TN/N : 85.00
Epoch [   3/   7] 
	train batch [  51/  51] - Loss : 0.0756 - TP/P : 100.00 - TN/N : 88.890
	test batch [  13/  13] - Loss : 0.0703 - TP/P : 90.00 - TN/N : 90.00
Epoch [   4/   7] 
	train batch [  51/  51] - Loss : 0.0362 - TP/P : 100.00 - TN/N : 88.890
	test batch [  13/  13] - Loss : 0.1296 - TP/P : 90.00 - TN/N : 85.00
Epoch [   5/   7] 
	train batch [  51/  51] - Loss : 0.0030 - TP/P : 100.00 - TN/N : 100.00
	test batch [  13/  13] - Loss : 0.1321 - TP/P : 90.00 - TN/N : 85.000
Epoch [   6/   7] 
	train batch [  51/  51] - Loss : 0.1336 - TP/P : 77.78 - TN/N : 88.8900
	test batch [  13/  13] - Loss : 0.0840 - TP/P : 70.00 - TN/N : 90.00
Epoch [   7/   7] 
	train bat

### Taux de bonnes prédiction dans train avec équilibre

In [162]:
net_eq.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(train_dataloader_eq):
  
  yhat = net_eq(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 98.21%
Taux de prediction sur les pos: 98.39%
Taux de prediction sur les neg: 98.02%
F1 score: 98.21%

accuracy score: 98.21%
precision score: 98.39%
recall score: 98.03%
f1 score: 98.21%


### Taux de bonnes prédiction dans train

In [ ]:
net_eq.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(train_dataloader):
  
  yhat = net_eq(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 73.38%
Taux de prediction sur les pos: 98.39%
Taux de prediction sur les neg: 72.84%
F1 score: 13.61%

accuracy score: 73.38%
precision score: 98.39%
recall score: 7.31%
f1 score: 13.61%


### Taux de bonnes prédiction dans test avec equilibre

In [163]:
net_eq.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(test_dataloader_eq):
  
  yhat = net_eq(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 80.45%
Taux de prediction sur les pos: 87.62%
Taux de prediction sur les neg: 73.27%
F1 score: 81.76%

accuracy score: 80.45%
precision score: 87.62%
recall score: 76.62%
f1 score: 81.76%


### Taux de bonnes prédiction dans test

In [ ]:
net_eq.eval()
lyhat, ly = [], []
for batch_idx, (batch, y) in enumerate(test_dataloader):
  
  yhat = net_eq(batch).squeeze(dim=1)
  ly.extend(y)
  lyhat.extend([1 if yihat > 0.5 else 0 for yihat in yhat])

lyhat, ly = torch.tensor(lyhat), torch.tensor(ly)

ipos, ineg = ly == 1, ly == 0
ly_pos, ly_neg = ly[ipos], ly[ineg]
lyhat_pos, lyhat_neg = lyhat[ipos], lyhat[ineg]

print(f'Taux de prediction total: {(100*((lyhat == ly).sum()/ len(ly))):.2f}%')
print(f'Taux de prediction sur les pos: {(100*((lyhat_pos == ly_pos).sum()/ len(ly_pos))):.2f}%')
print(f'Taux de prediction sur les neg: {(100*((lyhat_neg == ly_neg).sum()/ len(ly_neg))):.2f}%')
f1_loss = F1Loss()
print(f'F1 score: {(100*(1-f1_loss(ly, lyhat))):.2f}%')

# scores
tp = (lyhat_pos == ly_pos).sum()
tn = (lyhat_neg == ly_neg).sum()
fp = (lyhat_pos != ly_pos).sum()
fn = (lyhat_neg != ly_neg).sum()

accuracy_score = (tp + tn) / (tp + tn + fp + fn)
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2*precision_score*recall_score / (precision_score + recall_score)

print(f'\naccuracy score: {(100*accuracy_score):.2f}%')
print(f'precision score: {(100*precision_score):.2f}%')
print(f'recall score: {(100*recall_score):.2f}%')
print(f'f1 score: {(100*f1_score):.2f}%')


Taux de prediction total: 72.38%
Taux de prediction sur les pos: 87.62%
Taux de prediction sur les neg: 72.05%
F1 score: 11.90%

accuracy score: 72.38%
precision score: 87.62%
recall score: 6.38%
f1 score: 11.90%
